In [56]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings
import time
from sqlalchemy import create_engine, text


warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]
vsuccess_engine = create_engine(os.environ.get("VSUCCESS_URI"))

#### Lấy dữ liệu từ Mongo

In [57]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'all']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['hsA', 'hsB', 'hsC', 'hsD']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [58]:
def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

In [59]:
#Tạo dict map thời gian và số lượng cổ phiếu
quarter_map = pd.read_excel("../.xlsx/quarter_stock_map.xlsx", sheet_name='quarter_map')
quarter_map_dict = quarter_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

#Lấy ra list cổ phiếu của giai đoạn hiện tại
quarter_stock_list = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='quarter_stock_list')

current_quarter_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) 
                                  & set(quarter_stock_list[get_quarter('current_quarter')].dropna().tolist()))

total_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) & set(quarter_stock_list['all'].dropna().tolist()))

#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [quarter_map_dict['q2_2020'][0], quarter_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [quarter_map_dict[get_quarter('current_quarter')][0], quarter_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [quarter_map_dict[get_quarter('previous_quarter')][0], quarter_map_dict[get_quarter('previous_quarter')][1]]

In [60]:
def get_mongo_df(df_name, find_query=None, projection=None):
    # Truy cập collection
    collection = stock_db[df_name]
    # Nếu không truyền vào find_query thì mặc định lấy tất cả document
    if find_query is None:
        find_query = {}
    # Nếu không truyền vào projection thì mặc định loại bỏ trường _id
    if projection is None:
        projection = {"_id": 0}
    # Thực hiện lệnh find với điều kiện và projection đã cho
    docs = collection.find(find_query, projection)
    # Chuyển đổi kết quả sang DataFrame và trả về
    df = pd.DataFrame(list(docs))
    return df

In [61]:
#Tạo date_series cho thời gian tính toán
date_series = pd.read_excel("../.xlsx/date_series.xlsx", sheet_name="date_series")

#Xác định ngày hiện tại
today = date_series['date'].iloc[0]

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:30:00', freq='1T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 15:00:00', freq='1T'))

#Khởi tạo khung thời gian bắt đầu từ 9h00 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list).rename(columns={0:'date'}).sort_values('date', ascending=False).reset_index(drop=True)

In [63]:
# #Đọc dữ liệu lịch sử cổ phiếu
eod_stock_df = get_mongo_df("eod_stock", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1})
prev_stock_df = get_mongo_df("history_stock", find_query={"date": date_series.iloc[1].item()}, projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1})

#Đọc dữ liệu lịch sử cổ phiếu
eod_index_df = get_mongo_df("eod_index", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, 'option': 1, "t0_score": 1, "t5_score": 1})
history_index_df = get_mongo_df("history_index", find_query={"type": "spot"}, projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, 'option': 1, "t0_score": 1, "t5_score": 1})
full_index_df = pd.concat([eod_index_df, history_index_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu lịch sử cổ phiếu
itd_index_df = get_mongo_df("itd_index", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1})
#Đọc dữ liệu lịch sử cổ phiếu
itd_stock_df = get_mongo_df("itd_stock", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1})

#Đọc dữ liệu điểm dòng tiền nhóm cổ phiếu
eod_group_df = get_mongo_df("eod_group")

#Đọc dữ liệu điểm dòng tiền nhóm cổ phiếu
itd_group_df = get_mongo_df("itd_group")

#Đọc dữ liệu giao dịch của other
other_ticker_df = get_mongo_df("other_ticker")

#Đọc dữ liệu nntd_index
nntd_index_df = get_mongo_df("nntd_index")

#Đọc dữ liệu nntd_stock
nntd_stock_df = get_mongo_df("nntd_stock")

#Đọc dữ liệu market_phase
market_phase_df = get_mongo_df("market_phase")

#Đọc dữ liệu ms lịch sử
eod_ms_chart_df = get_mongo_df("eod_ms_chart")
history_ms_chart_df = get_mongo_df("history_ms_chart")
full_ms_chart_df = pd.concat([eod_ms_chart_df, history_ms_chart_df], axis=0).reset_index(drop=True)

In [64]:
#Tạo lại eod_index_dict từ dữ liệu monggoDB
eod_index_dict = {}
for ticker in full_index_df['ticker'].unique():
    temp_df = full_index_df[full_index_df['ticker'] == ticker].reset_index(drop=True)
    eod_index_dict[ticker] = temp_df

#Tạo lại itd_index_dict từ dữ liệu monggoDB
itd_index_dict = {}
for ticker in itd_index_df['ticker'].unique():
    temp_df = itd_index_df[itd_index_df['ticker'] == ticker].reset_index(drop=True)
    itd_index_dict[ticker] = temp_df

#Tạo lại itd_stock_dict từ dữ liệu monggoDB
itd_stock_dict = {}
for ticker in itd_stock_df['ticker'].unique():
    temp_df = itd_stock_df[itd_stock_df['ticker'] == ticker].reset_index(drop=True)
    itd_stock_dict[ticker] = temp_df

#Tạo lại itd_group_dict từ dữ liệu monggoDB
itd_group_dict = {}
for ticker in itd_group_df['ticker'].unique():
    temp_df = itd_group_df[itd_group_df['ticker'] == ticker].reset_index(drop=True)
    itd_group_dict[ticker] = temp_df

#Tạo lại other_dict từ dữ liệu monggoDB
other_dict = {}
for ticker in other_ticker_df['ticker'].unique():
    temp_df = other_ticker_df[other_ticker_df['ticker'] == ticker].reset_index(drop=True)
    other_dict[ticker] = temp_df

#### Tạo các bảng báo cáo

##### Chỉ số thị trường

In [65]:
world_index_history = date_series.copy()
for ticker, df in other_dict.items():
    if ticker in ['^BTC', '^DJI', '^XAU', 'CL=F', 'DXY']:
        world_index_history[f'{ticker}_value'] = df['close']
        world_index_history[f'{ticker}_volume'] = df['volume']
for ticker, df in eod_index_dict.items():
    if ticker == 'VNINDEX':
        world_index_history[f'{ticker}_value'] = df['close']
        world_index_history[f'{ticker}_volume'] = df['volume']

world_index_history = world_index_history.iloc[:60].sort_values('date')

In [66]:
world_index_volatility = pd.DataFrame()
for ticker, df in other_dict.items():
    if ticker in ['^BTC', '^DJI', '^XAU', 'CL=F', 'DXY']:
        temp_df = df.copy()
        temp_df['value_traded'] = 0
        temp_df['value_change'] = temp_df['close'][::-1].diff()[::-1].fillna(0)
        temp_df['pct_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
        temp_df['1w_change'] = temp_df['close'][::-1].rolling(window=5, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df['1m_change'] = temp_df['close'][::-1].rolling(window=20, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df['3m_change'] = temp_df['close'][::-1].rolling(window=60, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df = temp_df.iloc[[0]][['ticker','open','high','low','close','volume','value_traded','value_change','pct_change','1w_change','1m_change','3m_change']]

        world_index_volatility = pd.concat([world_index_volatility, temp_df], axis=0)

for ticker, df in eod_index_dict.items():
    if ticker == 'VNINDEX':
        temp_df = df.copy()
        temp_df['value_traded'] = temp_df['option']
        temp_df['value_change'] = temp_df['close'][::-1].diff()[::-1].fillna(0)
        temp_df['pct_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
        temp_df['1w_change'] = temp_df['close'][::-1].rolling(window=5, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df['1m_change'] = temp_df['close'][::-1].rolling(window=20, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df['3m_change'] = temp_df['close'][::-1].rolling(window=60, min_periods=1).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
        temp_df = temp_df.iloc[[0]][['ticker','open','high','low','close','volume','value_traded','value_change','pct_change','1w_change','1m_change','3m_change']]

        world_index_volatility = pd.concat([world_index_volatility, temp_df], axis=0)

##### Khối ngoại tự doanh

In [67]:
#Tạo bảng giá trị giao dịch ngày hiện tại
nn_index_df = nntd_index_df.copy()
nn_index_df = nn_index_df[(nn_index_df['date'] == today) & nn_index_df['ticker'].isin(['VNINDEX_NN', 'HNXINDEX_NN', 'UPINDEX_NN'])]
nn_index_df = nn_index_df.drop(columns=['date','type','ticker'])

td_index_df = nntd_index_df.copy()
td_index_df = td_index_df[(td_index_df['date'] == date_series['date'].iloc[1]) & td_index_df['ticker'].isin(['VNINDEX_TD', 'HNXINDEX_TD', 'UPINDEX_TD'])]
td_index_df = td_index_df.drop(columns=['date','type','ticker'])

nntd_value = pd.DataFrame({'NN': nn_index_df.sum(axis=0),'TD': td_index_df.sum(axis=0)}).reset_index()

In [68]:
#Bảng lịch sử giao dịch 10 phiên
nn_history_df = nntd_index_df.copy()
nn_history_df = nn_history_df[nn_history_df['ticker'].isin(['VNINDEX_NN', 'HNXINDEX_NN', 'UPINDEX_NN'])]
nn_history_df = nn_history_df.groupby('date')['net_value'].sum()

td_history_df = nntd_index_df.copy()
td_history_df = td_history_df[td_history_df['ticker'].isin(['VNINDEX_TD', 'HNXINDEX_TD', 'UPINDEX_TD'])]
td_history_df = td_history_df.groupby('date')['net_value'].sum()

#Ghép bảng và chỉ lấy 10 ngày gần nhất
nntd_history = pd.concat([nn_history_df, td_history_df], axis=1).reset_index().tail(10)
nntd_history.columns = ['date','NN','TD']

In [69]:
#Bảng top cổ phiếu giao dịch phiên hiện tại

#Tính toán cho nước ngoài
nn_stock_df = nntd_stock_df.copy()
nn_stock_df = nn_stock_df[(nn_stock_df['date'] == today) & nn_stock_df['ticker'].str.endswith('_NN')]
nn_stock_df['ticker'] = nn_stock_df['ticker'].apply(lambda x: x[:3])

top_nn_stock = nn_stock_df.sort_values('net_value', ascending=False).iloc[:5].reset_index(drop=True)
top_nn_stock_string = ", ".join([f"{r['ticker']} ({r['net_value']:.2f} tỷ)" for _, r in top_nn_stock.head(5).iterrows()])

bot_nn_stock = nn_stock_df.sort_values('net_value', ascending=True).iloc[:5].reset_index(drop=True)
bot_nn_stock_string = ", ".join([f"{r['ticker']} ({r['net_value']:.2f} tỷ)" for _, r in bot_nn_stock.head(5).iterrows()])

#Tính toán cho tự doanh
td_stock_df = nntd_stock_df.copy()
td_stock_df = td_stock_df[(td_stock_df['date'] == date_series['date'].iloc[1]) & td_stock_df['ticker'].str.endswith('_TD')]
td_stock_df['ticker'] = td_stock_df['ticker'].apply(lambda x: x[:3])

top_td_stock = td_stock_df.sort_values('net_value', ascending=False).iloc[:5].reset_index(drop=True)
top_td_stock_string = ", ".join([f"{r['ticker']} ({r['net_value']:.2f} tỷ)" for _, r in top_td_stock.head(5).iterrows()])

bot_td_stock = td_stock_df.sort_values('net_value', ascending=True).iloc[:5].reset_index(drop=True)
bot_td_stock_string = ", ".join([f"{r['ticker']} ({r['net_value']:.2f} tỷ)" for _, r in bot_td_stock.head(5).iterrows()])

nntd_top_stock = pd.DataFrame({"NN": [top_nn_stock_string, bot_nn_stock_string], "TD": [top_td_stock_string, bot_td_stock_string]}, index=["buy", "sell"]).reset_index()

##### Trạng thái thị trường

In [70]:
def add_count_columns(df):
    # Lấy danh sách các cột ban đầu để tính toán, tránh đếm cả các cột mới tạo
    original_columns = df.columns.tolist()
    
    df['positive_count'] = df[original_columns].gt(0).sum(axis=1) + df[original_columns].eq(0).sum(axis=1)
    df['negative_count'] = df[original_columns].lt(0).sum(axis=1)
    df['total_count']    = df[original_columns].count(axis=1)
    
    return df

itd_market_sentiment = itd_series.copy().set_index('date')
for ticker, df in itd_stock_dict.items():
    if ticker in current_quarter_stock_list:
        itd_market_sentiment[ticker] = df.set_index('date')['t0_score']

itd_market_sentiment = add_count_columns(itd_market_sentiment)
itd_market_sentiment['sentiment'] = (itd_market_sentiment['positive_count'] / itd_market_sentiment['total_count'])*100

itd_market_state = itd_index_dict['VNINDEX'].copy().set_index('date')
itd_market_state = itd_market_state.drop(columns=['ticker'])
itd_market_state['vol_ratio'] = itd_group_dict['all'].set_index('date')['vol_ratio']
itd_market_state['sentiment'] = itd_market_sentiment['sentiment']

itd_market_state = itd_market_state.sort_index().reset_index().dropna()
itd_market_state['last_vol_ratio'] = itd_market_state['vol_ratio'].iloc[-1]
itd_market_state['quote_vol_ratio'] = itd_market_state['last_vol_ratio'].apply(lambda x: 'Rất thấp' if x < 0.5 else (
                                                                            'Thấp' if (x >= 0.5) & (x < 0.8) else (
                                                                            'Trung bình' if (x >= 0.8) & (x < 1.2) else (
                                                                            'Cao' if (x >= 1.2) & (x < 1.5) else 'Rất cao'))))

itd_market_state['last_sentiment'] = itd_market_state['sentiment'].iloc[-1]
itd_market_state['quote_sentiment'] = itd_market_state['last_sentiment'].apply(lambda x: 'Sợ hãi' if x < 20 else
                                                                ('Tiêu cực' if (x >= 20) & (x < 40) else
                                                                ('Trung lập' if (x >= 40) & (x < 60) else
                                                                ('Tích cực' if (x >= 60) & (x < 80) else 'Hưng phấn'))))

##### Dòng tiền thị trường

In [71]:
# Tạo bảng để vẽ biểu đồ tròn
market_score = itd_market_sentiment.dropna()[['positive_count','negative_count']]
market_score.columns = ['Dòng tiền dương', 'Dòng tiền âm']
market_score = market_score.iloc[[0]].reset_index(drop=True)
market_score = market_score.transpose().reset_index()
market_score.columns = ['index','value']

temp_industry_name_df = eod_group_df[eod_group_df['ticker'].isin(industry_name_list)].sort_values('t0_score', ascending=False)
temp_industry_name_df = temp_industry_name_df[['ticker', 't0_score']]
temp_industry_name_df['ticker'] = temp_industry_name_df['ticker'].map(name_map_dict)
temp_industry_name_df.columns = ['index','value']

temp_industry_perform_df = eod_group_df[eod_group_df['ticker'].isin(industry_perform_list)].sort_values('t0_score', ascending=False)
temp_industry_perform_df = temp_industry_perform_df[['ticker', 't0_score']]
temp_industry_perform_df['ticker'] = temp_industry_perform_df['ticker'].map(name_map_dict)
temp_industry_perform_df.columns = ['index','value']

temp_marketcap_group_df = eod_group_df[eod_group_df['ticker'].isin(marketcap_group_list)].sort_values('t0_score', ascending=False)
temp_marketcap_group_df = temp_marketcap_group_df[['ticker', 't0_score']]
temp_marketcap_group_df['ticker'] = temp_marketcap_group_df['ticker'].map(name_map_dict)
temp_marketcap_group_df.columns = ['index','value']

market_score = pd.concat([market_score, temp_industry_name_df, temp_industry_perform_df, temp_marketcap_group_df], axis=0)

In [72]:
def calculate_pct_change(eod_stock_df, prev_stock_df):
    eod_df = eod_stock_df.copy().set_index('ticker')
    prev_df = prev_stock_df.copy().set_index('ticker')
    eod_df['pct_change'] = (eod_df['close'] - prev_df['close'])/prev_df['close']
    eod_df = eod_df.reset_index()

    return eod_df[['ticker','pct_change']]

market_stock_df = eod_stock_df.merge(calculate_pct_change(eod_stock_df, prev_stock_df), on='ticker', how='left')
temp_top_stock = market_stock_df.sort_values('t0_score', ascending=False).head(10)
temp_top_stock['type'] = 'Tích cực'
temp_bot_stock = market_stock_df.sort_values('t0_score', ascending=True).head(10)
temp_bot_stock['pct_change'] = temp_bot_stock['pct_change'].abs()
temp_bot_stock['type'] = 'Tiêu cực'

market_top_stock = pd.concat([temp_top_stock, temp_bot_stock], axis=0)
market_top_stock = market_top_stock[['type', 'ticker', 'pct_change']]

##### Giai đoạn thị trường

In [73]:
temp_ms_df = full_ms_chart_df[full_ms_chart_df['ticker']=='all']
temp_ms_df = temp_ms_df[['date','trend_5p','trend_20p','trend_60p','trend_120p','trend_240p','trend_480p']].set_index('date')
temp_vnidex_df = eod_index_dict['VNINDEX'][['date','close']].set_index('date')
temp_phase_df = market_phase_df[['date', 'market_phase_final']].set_index('date')
temp_phase_df.columns = ['market_phase']

market_phase_history = pd.concat([temp_vnidex_df, temp_phase_df, temp_ms_df], axis = 1)
market_phase_history = market_phase_history.iloc[:80].sort_index().reset_index()

In [74]:
market_phase_quote = market_phase_history.iloc[[-1]]
market_phase_quote['phase_name'] = market_phase_quote['market_phase'].apply(lambda x: 'An toàn' if x == 1 else "Rủi ro")

#### Lưu vào MSSQL

In [75]:
update_time = pd.DataFrame({'today': [f"Ngày {itd_market_state['date'].max().strftime('%d/%m/%Y %H:%M')}"]})

#Các bảng cho báo cáo hàng ngày
update_time.to_sql('update_time', vsuccess_engine, if_exists='replace', index=False)
world_index_history.to_sql('world_index_history', vsuccess_engine, if_exists='replace', index=False)
world_index_volatility.to_sql('world_index_volatility', vsuccess_engine, if_exists='replace', index=False)
nntd_value.to_sql('nntd_value', vsuccess_engine, if_exists='replace', index=False)
nntd_history.to_sql('nntd_history', vsuccess_engine, if_exists='replace', index=False)
nntd_top_stock.to_sql('nntd_top_stock', vsuccess_engine, if_exists='replace', index=False)
itd_market_state.to_sql('itd_market_state', vsuccess_engine, if_exists='replace', index=False)
market_score.to_sql('market_score', vsuccess_engine, if_exists='replace', index=False)
market_top_stock.to_sql('market_top_stock', vsuccess_engine, if_exists='replace', index=False)
market_phase_history.to_sql('market_phase_history', vsuccess_engine, if_exists='replace', index=False)
market_phase_quote.to_sql('market_phase_quote', vsuccess_engine, if_exists='replace', index=False)
market_stock_df.to_sql('market_stock_df', vsuccess_engine, if_exists='replace', index=False)

time.sleep(120)